# analysis
using duckdb native connection with jupysql. Jupytsql converts code cell into a sql cell.


In [208]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [209]:
import duckdb
import pandas as pd
import json
from pathlib import Path

In [210]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [211]:
conn = duckdb.connect("file.db")
%sql conn --alias duckdb

In [212]:
raw = pd.read_json(FILE, lines=True)

In [214]:
def unpack_summary(lst):
    """Turn the 14-item summary list into a dict with nice keys."""
    return {
        "overall_place": lst[1].rstrip(".") if lst[1].strip() else None,
        "bib":           lst[2],
        "name":          lst[3].title(),         # "STAGG, TOM" -> "Stagg, Tom"
        "category":      lst[5],                 # e.g. "Male 45-49"
        "club":          lst[6] or None,
        "finish_time":   lst[7].strip() or None,                
        "gap_overall":   lst[8].strip() or None,
        "gap_prev":      lst[9].strip() or None,
    }
    
def details_to_df(details, bib):
    header, *rows = details
    df = pd.DataFrame(rows, columns=header)

    df.insert(0, "bib", bib)

    # numeric cols
    for col in ["Overall", "Gender", "Age Group"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    for col in ["Gun Time", "Sector", "Time of Day"]:
        df[col] = df[col].replace(r"^\s*$", pd.NA, regex=True)   # blanks → <NA>
        
    # Timedelta columns
    df["Gun Time"] = pd.to_timedelta("00:" + df["Gun Time"], errors="coerce")
    df["Sector"]   = pd.to_timedelta("00:" + df["Sector"],   errors="coerce")

    # datetime column
    df["Time of Day"] = pd.to_datetime(
        df["Time of Day"], 
        format="%H:%M:%S",
        errors="coerce")
    return df

In [215]:
## read data
results_file = Path("../results.json") 
raw = pd.read_json(results_file, lines=True)


# process summary 
summary_df = raw["summary"].apply(unpack_summary).apply(pd.Series)

## tidy up dtypes
summary_df["finish_time"] = (
    pd.to_timedelta(
        summary_df["finish_time"]
            .replace(r"^\s*$", pd.NA, regex=True),
        errors ="coerce"
    )
)

# create human readable finish time
summary_df["finish_hms"] = (
    summary_df["finish_time"]
        .dt.round("s")
        .astype(str)
        .str.replace(r"^0 days\s+", "", regex=True)
)

# populated checkpoints_df using bib and details
checkpoints_df = pd.concat(
    (
        details_to_df(row.details, row.summary[2]) 
        for _, row in raw.iterrows()
    ),
    ignore_index=True
)


In [216]:
%%sql
create table summary as select * from summary_df;
create table checkpoints as select * from checkpoints_df

,Count
0,45474


In [184]:
summary_df.dtypes, checkpoints_df.dtypes

(overall_place             object
 bib                       object
 name                      object
 category                  object
 club                      object
 finish_time      timedelta64[ns]
 gap_overall               object
 gap_prev                  object
 finish_hms                object
 dtype: object,
 bib                     object
                         object
 Time of Day     datetime64[ns]
 Gun Time       timedelta64[ns]
 Sector         timedelta64[ns]
 Speed/Pace              object
 Overall                float64
 Gender                 float64
 Age Group              float64
 dtype: object)

In [217]:
conn.close()

In [221]:
checkpoints_df[checkpoints_df['bib']=='1677']

,bib,,Time of Day,Gun Time,Sector,Speed/Pace,Overall,Gender,Age Group
1378,1677,Startline,1900-01-01 09:29:02,0 days 00:00:02,NaT,,60.0,52.0,5.0
1379,1677,CP1,1900-01-01 09:35:16,0 days 00:06:16,0 days 00:06:15,22.1 km/h,397.0,383.0,48.0
1380,1677,CP2,1900-01-01 09:47:56,0 days 00:18:56,0 days 00:12:40,22.2 km/h,142.0,142.0,3.0
1381,1677,CP3,1900-01-01 10:02:41,0 days 00:33:41,0 days 00:14:45,34.9 km/h,122.0,122.0,9.0
1382,1677,CP4,1900-01-01 10:08:28,0 days 00:39:28,0 days 00:05:47,36.3 km/h,117.0,117.0,9.0
1383,1677,CP5,1900-01-01 10:20:36,0 days 00:51:36,0 days 00:12:08,26.7 km/h,107.0,107.0,6.0
1384,1677,CP6,1900-01-01 10:27:49,0 days 00:58:49,0 days 00:07:13,38.2 km/h,110.0,110.0,2.0
1385,1677,CP7,1900-01-01 10:34:21,NaT,0 days 00:06:32,29.3 km/h,99.0,99.0,1.0
1386,1677,CP8,1900-01-01 10:43:59,NaT,0 days 00:09:38,37.9 km/h,104.0,104.0,8.0
1387,1677,CP9,1900-01-01 10:54:10,NaT,0 days 00:10:11,28.2 km/h,91.0,91.0,4.0


In [223]:
summary_df

,overall_place,bib,name,category,club,finish_time,gap_overall,gap_prev,finish_hms
0,1,53,"Vakoc, Petr",Elite Men,Canyon Isadore,0 days 03:08:15.590000,--,--,03:08:16
1,2,51,"Holmes, Matthew",Elite Men,OGT Factor Racing,0 days 03:08:15.620000,+0:00,+0:00,03:08:16
2,3,107,"Perry, Benjamin",Elite Men,independent,0 days 03:08:16.200000,+0:00,+0:00,03:08:16
3,4,115,"Thomas, Ben",Elite Men,Ribble Outliers,0 days 03:08:16.850000,+0:01,+0:00,03:08:17
4,5,86,"Howell, Adam",Elite Men,MUC-OFF SRCT STORCK,0 days 03:08:49.240000,+0:33,+0:32,03:08:49
...,...,...,...,...,...,...,...,...,...
1744,DNF,3209,"Vallance, Alan",Male 65-69,Coalvill Wheelers CC,NaT,None,None,NaT
1745,DNS,1128,"Bruton, Richard",Male 40-44,,0 days 04:47:42.100000,None,None,04:47:42
1746,DNS,3012,"Mccann, David",Male 60-64,VC Deal,0 days 04:09:41.040000,None,None,04:09:41
1747,251,1296,"Simpson, Chris",Male 40-44,,0 days 03:53:47.160000,+45:31,+0:03,03:53:47
